In [1]:
from assets.models import Data
import pandas as pd
import datetime

from assets.database import db_session

In [2]:
df = pd.read_csv('assets/data.csv')

In [3]:
df.head()

,date,subscribers,reviews
0,2019/1/17,2671,194
1,2019/1/18,2671,195
2,2019/1/19,2674,197
3,2019/1/20,2685,199
4,2019/1/21,2695,199


date型への変換

In [4]:
df.loc[0, 'date']

'2019/1/17'

In [5]:
datetime.datetime.strptime(df.loc[0, 'date'], '%Y/%m/%d').date()

datetime.date(2019, 1, 17)

for文で記述する場合

In [6]:
for index, _df in df.iterrows():
    date = datetime.datetime.strptime(_df['date'], '%Y/%m/%d').date()
    row = Data(date=date, subscribers=_df.subscribers, reviews=_df.reviews)
    db_session.add(row)
# db_session.commit()
db_session.rollback()

リスト内包表記で記述する場合

In [7]:
to_date = (lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').date())
db_session.add_all([Data(date=to_date(row.date), subscribers=row.subscribers, reviews=row.reviews) for i, row in df.iterrows()])
# db_session.commit()
db_session.rollback()

メソッド化

In [8]:
def read_data():
    from assets import models
    df = pd.read_csv('assets/data.csv')
    to_date = (lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').date())
    db_session.add_all([Data(date=to_date(row.date), subscribers=row.subscribers, reviews=row.reviews) for i, row in df.iterrows()])
    db_session.commit()

In [9]:
from assets.database import init_db 

init_db()
read_data()

2019-05-27 06:39:08,221 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-27 06:39:08,222 INFO sqlalchemy.engine.base.Engine ()
2019-05-27 06:39:08,223 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-27 06:39:08,225 INFO sqlalchemy.engine.base.Engine ()
2019-05-27 06:39:08,227 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("data")
2019-05-27 06:39:08,229 INFO sqlalchemy.engine.base.Engine ()
2019-05-27 06:39:08,233 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE data (
	id INTEGER NOT NULL, 
	date DATE, 
	subscribers INTEGER, 
	reviews INTEGER, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2019-05-27 06:39:08,235 INFO sqlalchemy.engine.base.Engine ()
2019-05-27 06:39:08,247 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-27 06:39:08,289 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-27 06:39:08,291 INFO sqlalchemy.engine.base.Engine INSERT INTO data (date, su